# 📋 Ejercicio 4

> Escenifique compartir el secreto de valor 113132 entre 50 partícipes, requiriéndose el acuerdo de 42 de ellos para explicitar dicho secreto. Use en este ejercicio el esquema de Shamir de intercambio de secretos, pudiendo el alumno servirse de Sagemath implementando su propio software

Como se nos pide, vamos a ejemplificar el esquema de Shamir. 

Para empezar, escribamos nuestras hipótesis, usando la nomenclatura de la teoría:

In [ ]:
# Secreto
s = 113132 

# Número de participantes
n = 50

# Umbral de participantes para recuperar el secreto
t = 42

Para operar, debemos realizar los siguientes pasos:

1. Se elige un número primo $p$ tal que $p > máx\{n, s\}$
2. Se eligen $t - 1$ elementos de $Z_p$, $a_1, \dotsc, a_{t-1}$ de forma que $a_{t-1} \ne 0$
3. Consideramos el polinomio $p(x) = s + a_1x + \dotsc + a_{t-1}x^{t-1}$.
4. Son escogidos $x_1, \dotsc, x_n \in Z_p^*$, y se calcula para todo $1 \le i \le n$ el valor $y_i = p(x_i)$
5. El partícipe $i, 1 \le i \le n$, recibe la parte del secreto $<x_i, y_i>$ y la mantendrá como par secreto

Sin más dilación, vamos a ello:

In [ ]:
# ──────────────────────────────────────────────────────────────────────── 1 ─────

p = random_prime(2**64)

while p <= max(n, s):
    p = random_prime(2**64)

print(f'El primo generado es p = {p}. \n\t -> ¿Es mayor que {n} y {s}? {p > max(n, s)}\n')


# Definimos los anillos necesarios para operar
Zp = GF(p)
K.<x> = PolynomialRing(Zp)



# ──────────────────────────────────────────────────────────────────────── 2 ─────

A = [Zp.random_element() for i in range(0, t-1)]
print(f'Para A, hemos escogido {len(A)} = t-1 = {t-1} elementos de Zp\n')


# Nos aseguramos que A[t-2] = a_{t-1} != 0
while (A[t-2] == 0):
    A[t-2] = Zp.random_element()
    
print(f'Se tiene además que A[t-2] = a_(t-1) = {A[-1]} != 0\n')



# ──────────────────────────────────────────────────────────────────────── 3 ─────

p = K(A)*x + s
print(f'Escogemos el polinomio \n{p}\n\n')



# ──────────────────────────────────────────────────────────────────────── 4 ─────

X = [Zp.random_element() for i in range(0, n)]

# Limpiamos los posibles 0 de X
while True:
    try:
        i = X.index(0)
        X[i] = Zp.random_element()
    except ValueError:
        # Si no hay 0s, paramos
        break
    
Y = [p(x) for x in X]



# ──────────────────────────────────────────────────────────────────────── 5 ─────

pares_secretos = [ (x,y) for x, y in zip(X,Y) ]

print(f'Hemos obtenido {len(pares_secretos)} pares secretos')

Mostremos ahora la recuperación del secreto, escogemos $t$ participantes al azar:

In [ ]:
import random 
random.shuffle(pares_secretos)

estan_de_acuerdo = pares_secretos[:t]

print(f'Estos son los pares de las {t} personas que están de acuerdo:')

estan_de_acuerdo

Y ahora, estas personas que están de acuerdo, llevan a cabo la interpolación de Lagrange. Podría usarse el de Newton, pero teniendo en cuenta que el de Lagrange se ejecuta lo suficientemente rápido, lo escogeremos para esta prueba. Sage proporciona una manera rápida de obtenerlo que nos resulta ventajosa.

De esta forma, se descubre el secreto:

In [ ]:
q = K.lagrange_polynomial(estan_de_acuerdo)
q(0)

¿Qué ocurriría si no se llegara a la cantidad de personas requerida? 

In [ ]:
random.shuffle(pares_secretos)

frustrados = pares_secretos[:randint(1, t-1)]
r = K.lagrange_polynomial(frustrados)
r(0)

No son capaces de conseguir el secreto 🙈.